## Load Libraries

In [1]:
!pip install openpyxl
!pip install datasets
!pip install "transformers[torch]"

## Load Excel file containing labels

In [2]:
import pandas as pd
path1 = '../data/2025-05-01 Capstone Metadata.xlsx'
path2 = '../data/2025-05-13 additional Capstone Metadata.xlsx'

In [3]:
df1 = pd.read_excel(path1, header=3)
df1 = df1[['Document Type', 'Title/Subject']]
df1

,Document Type,Title/Subject
0,CORR,"Standards, Analytical Results"
1,COV,Covenant
2,CORR,"Certificate of Compliance, Pursuant to Section..."
3,CoC,Conditional Certificate of Compliance
4,CoC,Conditional Certificate of Compliance
...,...,...
220,RPT,Site Remediation Report\nNorth Kamloops Fire H...
221,RPT,"Underground Storage Tank Decommissioning,\nSup..."
222,RPT,Geotechnical Investigation\nProposed Senior Ci...
223,PSI,STAGE 2 PSI\nCITY OF KAMLOOPS FIREHALL NO. 2


In [4]:
df2 = pd.read_excel(path2, header=3)
df2 = df2[['Document Type', 'Title/Subject']]
df2

,Document Type,Title/Subject
0,FDET,RE: Final Determination pursuant to Section 26...
1,FDET,Re: Final Determination Pursuant to Section 26...
2,FDET,Re: Final Determination Pursuant to Section 26...
3,FDET,Re: Final Determination Pursuant to Section 26...
4,FDET,Re: Final Determination Pursuant to Section 26...
5,PDET,Re: Preliminary Determination Pursuant to Sect...
6,PDET,Re: Preliminary Determination Pursuant to Sect...
7,PDET,Re: Preliminary Determination Pursuant to Sect...
8,PDET,Re: Preliminary Determination Pursuant to Sect...
9,PDET,Re: Preliminary Determination Pursuant to Sect...


In [5]:
df = pd.concat([df1, df2], ignore_index=True)
df

,Document Type,Title/Subject
0,CORR,"Standards, Analytical Results"
1,COV,Covenant
2,CORR,"Certificate of Compliance, Pursuant to Section..."
3,CoC,Conditional Certificate of Compliance
4,CoC,Conditional Certificate of Compliance
...,...,...
251,AiP,APPROVAL IN PRINCIPLE
252,DSI,DETAILED SITE INVESTIGATION REPORT\nFORMER HAR...
253,AiP,APPROVAL IN PRINCIPLE
254,DSI,DETAILED SITE INVESTIGATION\nVOLUME II\nSUPPOR...


In [6]:
df['Document Type'].value_counts()

Document Type
CORR             103
RPT               19
TMEMO             17
PSI               12
DSI               11
FDET              10
TITLE             10
PDET               8
AiP                8
MAP                7
Site Registry      6
COC                6
RA                 5
CSSA               5
NIR                5
NOTE               4
CoC                4
NOM                3
REF                2
NIRI               2
SSI                2
AIP                2
COA                1
COV                1
SPC                1
SP                 1
IMG                1
Name: count, dtype: int64

## Some names were not consistent. Make them consistent

In [7]:
rename_map = {
    'CoC' : 'COC',
    'AiP' : 'AIP',
    'NOTE': 'OTHERS',
    'REF': 'OTHERS',
    'SPC': 'OTHERS'
}

df['Document Type'] = df['Document Type'].replace(rename_map)

In [8]:
df['Document Type'].value_counts()

Document Type
CORR             103
RPT               19
TMEMO             17
PSI               12
DSI               11
TITLE             10
COC               10
AIP               10
FDET              10
PDET               8
OTHERS             7
MAP                7
Site Registry      6
CSSA               5
RA                 5
NIR                5
NOM                3
NIRI               2
SSI                2
IMG                1
COV                1
SP                 1
COA                1
Name: count, dtype: int64

## Data Augmentation to make sure minority classes are not underrepresented

In [9]:
new_row = pd.DataFrame([
        {'Document Type': 'COV', 
        'Title/Subject': '''Covenant'''},
            {'Document Type': 'NOM', 
        'Title/Subject': '''This is to notify you that potential substances have migrated to your property'''},
    {'Document Type': 'NOM', 
        'Title/Subject': '''Potential substances migrated to your property notification'''},
             {'Document Type': 'NOM', 
        'Title/Subject': ''''Migration Of Contamination To Neighbouring Property From 97 Victoria'''},
        {'Document Type': 'NOM', 
        'Title/Subject': ''''Migration Of Contamination To Neighbouring Property From 24 Mount Pleasant, Vancouver, BC, PID 236-604-174'''},
    {'Document Type': 'NOM', 
        'Title/Subject': ''''Migration Of Contamination To Neighbouring Property From 465 Shambala BC PID 604-198-174'''},
            {'Document Type': 'COA', 
        'Title/Subject': '''Certifi cate of Analysis for document of Site 3452'''},
        {'Document Type': 'COA', 
        'Title/Subject': '''Certificate of Analysis Document no 3425'''},
    {'Document Type': 'COA', 
        'Title/Subject': '''Certificate of\n\n Analysis done completely'''},
        {'Document Type': 'COA', 
        'Title/Subject': '''Certificate of completed Analysis'''},
            {'Document Type': 'IMG', 
        'Title/Subject': '''Pictures'''},
    {'Document Type': 'IMG', 
        'Title/Subject': '''Images'''},
    {'Document Type': 'IMG', 
        'Title/Subject': '''JPEGS'''},
    {'Document Type': 'IMG', 
        'Title/Subject': '''PNGS'''},
    {'Document Type': 'COV', 
        'Title/Subject': '''Cov\enant'''},
    {'Document Type': 'COV', 
        'Title/Subject': '''Covenant Report'''},
    {'Document Type': 'COV', 
        'Title/Subject': '''Covenant Based on '''},
    {'Document Type': 'SP', 
        'Title/Subject': '''Schedule 2 Site Profile'''},
        {'Document Type': 'SP', 
        'Title/Subject': '''Schedule 3 Site Profile'''},
      {'Document Type': 'SP', 
        'Title/Subject': '''Schedule 4 Site Profile'''},
    {'Document Type': 'SP', 
        'Title/Subject': '''Schedule 5 Site Profile'''},
    {'Document Type': 'PDET', 
        'Title/Subject': '''preliminary\ndetermination for Site 2345'''},
    {'Document Type': 'SSI', 
        'Title/Subject': '''STAGE 1 AND 2 PSI, DSI AND CLOSURE REP\nORTING\n'''},
    {'Document Type': 'SSI', 
        'Title/Subject': '''STAGE 1 AND 2 PSI, DSI AND CLOSURE REPORTING ADDITIONAL SUPPLEMENTAL INFORMATION (given on page 32)'''},
    {'Document Type': 'SSI', 
        'Title/Subject': '''STAGE 1 AND 2 PSI, DSI AND CLOSURE REPORTING SUPPLEMENTAL INFORMATION OR ADDITIONAL\n'''},
    {'Document Type': 'AIP', 
        'Title/Subject': '''APPROVAL In Principle, technical report for PA 16456'''},
    {'Document Type': 'AIP', 
        'Title/Subject': '''Approval in Principle, AIP 23663'''},
    {'Document Type': 'PDET', 
        'Title/Subject': '''Preliminary Determination for Section 26.4  for the property\nlocated at 736 Main Street'''},
    {'Document Type': 'NIRI', 
    'Title/Subject': '''Notice of Independent Remediation  
    Vancouver, British Columbia'''}, 
    {'Document Type': 'NIRI', 
        'Title/Subject': '''Notice of Independent Remediation'''},
        {'Document Type': 'NIRI', 
    'Title/Subject': '''RE: NOTICE OF INDEP ENDENT REMEDIA\nTION\tMOBILE PHONE NO. 72 STREET, SHAMBALA, BC\n'''}])
df = pd.concat([df, new_row], ignore_index=True)

In [10]:
df['Document Type'].value_counts()

Document Type
CORR             103
RPT               19
TMEMO             17
AIP               12
PSI               12
DSI               11
TITLE             10
COC               10
FDET              10
PDET              10
NOM                8
MAP                7
OTHERS             7
Site Registry      6
COV                5
IMG                5
SP                 5
CSSA               5
COA                5
NIRI               5
RA                 5
SSI                5
NIR                5
Name: count, dtype: int64

In [11]:
df['Document Type'].unique()

array(['CORR', 'COV', 'COC', 'TMEMO', 'RA', 'FDET', 'PDET', 'CSSA', 'SP',
       'OTHERS', 'MAP', 'Site Registry', 'TITLE', 'IMG', 'RPT', 'PSI',
       'DSI', 'COA', 'NIRI', 'AIP', 'SSI', 'NIR', 'NOM'], dtype=object)

## Train, test, val splits

In [12]:
import pandas as pd
from math import ceil

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Containers for splits
train_rows, val_rows, test_rows = [], [], []

# Per-class splitting
for label, group in df.groupby('Document Type'):
    n = len(group)
    n_train = int(n * 0.6)
    n_val = int(n * 0.2)
    n_test = n - n_train - n_val  # ensures all rows are used

    train_rows.append(group.iloc[:n_train])
    val_rows.append(group.iloc[n_train:n_train + n_val])
    test_rows.append(group.iloc[n_train + n_val:])

# Concatenate all rows
train_df = pd.concat(train_rows).reset_index(drop=True)
val_df = pd.concat(val_rows).reset_index(drop=True)
test_df = pd.concat(test_rows).reset_index(drop=True)

## Prepare data for training 

In [13]:
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder


train_df = train_df.rename(columns={'Title/Subject': 'text'})
train_df['text'] = train_df['text'].astype(str).fillna('')

val_df = val_df.rename(columns={'Title/Subject': 'text'})
test_df = test_df.rename(columns={'Title/Subject': 'text'})

# Label encode target
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['Document Type'])
val_df['label'] = le.transform(val_df['Document Type'])
test_df['label'] = le.transform(test_df['Document Type'])

# Create Hugging Face Datasets
train_ds = Dataset.from_pandas(train_df[['text', 'label']])
val_ds = Dataset.from_pandas(val_df[['text', 'label']])
test_ds = Dataset.from_pandas(test_df[['text', 'label']])

/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    texts = list(batch['text'])  # force list in case it's not
    return tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=64
    )



train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

Map: 100%|████████████████████████████| 66/66 [00:00<00:00, 15687.64 examples/s]


## Load Model 

In [15]:
from transformers import AutoModelForSequenceClassification

from sklearn.utils.class_weight import compute_class_weight

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"


import torch
import numpy as np

device = torch.device("cpu")


class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)

# Convert to torch tensor
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights = class_weights.to(device)


num_labels = len(le.classes_)
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=num_labels
)
model.to(device)

# model.classifier.weight = torch.nn.Parameter(model.classifier.weight * class_weights.unsqueeze(1))

/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/deepaksirwani/miniforge3/envs/colx563/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvis

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [16]:
train_ds.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_ds.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

## Loss function

In [17]:
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss(weight=class_weights)

## Train the model 

In [18]:
from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     save_strategy='epoch',
#     load_best_model_at_end=True,
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=32,
#     num_train_epochs=50,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
#     no_cuda=True
# )

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",              
    load_best_model_at_end=True,        
    save_total_limit=1,                 
    save_steps=0,                       
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=50,
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-5,
)

# from transformers import Trainer

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_ds,
#     eval_dataset=val_ds,
#     tokenizer=tokenizer,
# )
# trainer.train()


from transformers import Trainer

# class FocalLossTrainer(Trainer):
#     def __init__(self, focal_loss_fn, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.focal_loss_fn = focal_loss_fn

#     def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
#         labels = inputs.pop("labels")
    
#         # Force CPU
#         device = torch.device("cpu")
#         labels = labels.to(device)
#         model.to(device)
#         for k in inputs:
#             if isinstance(inputs[k], torch.Tensor):
#                 inputs[k] = inputs[k].to(device)

#         outputs = model(**inputs)
#         logits = outputs.logits
#         loss = self.focal_loss_fn(logits, labels)
#         return (loss, outputs) if return_outputs else loss

class WeightedCETrainer(Trainer):
    def __init__(self, loss_fn, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fn = loss_fn

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        labels = labels.to(logits.device)
        self.loss_fn.weight = self.loss_fn.weight.to(logits.device)
        loss = self.loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss



# focal_loss = WeightedFocalLoss(alpha=class_weights, gamma=1.0)

# trainer = FocalLossTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_ds,
#     eval_dataset=val_ds,
#     tokenizer=tokenizer,
#     focal_loss_fn=focal_loss
# )

trainer = WeightedCETrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    loss_fn=loss_fn
)


trainer.train()

/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/xx/mk_s8q213n37jfd4755v6dfr0000gn/T/ipykernel_3172/612370784.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedCETrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,3.133300,3.096269
2,3.075000,3.019091
3,2.956100,2.915319
4,2.848300,2.780320
5,2.715900,2.644043
6,2.538000,2.486252
7,2.396900,2.351433
8,2.297500,2.197642
9,2.184900,2.087157
10,1.935000,1.977668


TrainOutput(global_step=1100, training_loss=0.5214474860375578, metrics={'train_runtime': 317.4057, 'train_samples_per_second': 52.929, 'train_steps_per_second': 3.466, 'total_flos': 278285717606400.0, 'train_loss': 0.5214474860375578, 'epoch': 100.0})

## Get performance on test data

In [19]:
preds = trainer.predict(test_ds)
pred_labels = preds.predictions.argmax(axis=-1)
from sklearn.metrics import classification_report
print(classification_report(test_ds['label'], pred_labels, target_names=le.classes_))

               precision    recall  f1-score   support

          AIP       1.00      1.00      1.00         3
          COA       1.00      1.00      1.00         1
          COC       1.00      1.00      1.00         2
         CORR       0.77      0.91      0.83        22
          COV       0.00      0.00      0.00         1
         CSSA       0.00      0.00      0.00         1
          DSI       1.00      1.00      1.00         3
         FDET       0.67      1.00      0.80         2
          IMG       0.50      1.00      0.67         1
          MAP       0.00      0.00      0.00         2
          NIR       1.00      1.00      1.00         1
         NIRI       1.00      1.00      1.00         1
          NOM       1.00      1.00      1.00         3
       OTHERS       0.00      0.00      0.00         2
         PDET       1.00      1.00      1.00         2
          PSI       1.00      1.00      1.00         3
           RA       0.50      1.00      0.67         1
         

/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/deepaksirwani/miniforge3/envs/colx563/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Save the best model 

In [20]:
le.classes_

array(['AIP', 'COA', 'COC', 'CORR', 'COV', 'CSSA', 'DSI', 'FDET', 'IMG',
       'MAP', 'NIR', 'NIRI', 'NOM', 'OTHERS', 'PDET', 'PSI', 'RA', 'RPT',
       'SP', 'SSI', 'Site Registry', 'TITLE', 'TMEMO'], dtype=object)

In [21]:
trainer.save_model("./best_model")               # saves model weights + config
tokenizer.save_pretrained("./best_model")        # saves tokenizer (needed for inference)

('./best_model/tokenizer_config.json',
 './best_model/special_tokens_map.json',
 './best_model/vocab.txt',
 './best_model/added_tokens.json',
 './best_model/tokenizer.json')

## Inference

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('./best_model')
model = AutoModelForSequenceClassification.from_pretrained('./best_model')
model.eval()

# Replace with your actual class names
class_names = ['AIP', 'COA', 'COC', 'CORR', 'COV', 'CSSA', 'DSI', 'FDET', 'IMG',
       'MAP', 'NIR', 'NIRI', 'NOM', 'OTHERS', 'PDET', 'PSI', 'RA', 'RPT',
       'SP', 'SSI', 'Site Registry', 'TITLE', 'TMEMO']

# Prediction function
def predict_document_type(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return class_names[predicted_class_id]

# Example usage
example_titles = [
    "Certificate of Compliance for Section 28",
    "Preliminary Site Investigation for North Shore",
    "Photo of Soil Contamination",
    "Analytical Lab Results",
    "covenant report sent to you",
    "potential substances have contaminated your property"
]

predictions = [predict_document_type(title) for title in example_titles]
for title, label in zip(example_titles, predictions):
    print(f"{title} → {label}")

Certificate of Compliance for Section 28 → COC
Preliminary Site Investigation for North Shore → PSI
Photo of Soil Contamination → CORR
Analytical Lab Results → CORR
covenant report sent to you → COV
potential substances have contaminated your property → NOM
